In [1]:
import torch
import torch.nn as nn
from models import GridMLMMelHarm
from GridMLM_tokenizers import CSGridMLMTokenizer
from data_utils import CSGridMLMDataset, CSGridMLM_collate_fn
from torch.utils.data import DataLoader
from train_utils import apply_masking
from generate_utils import generate_files_with_base2, generate_files_with_base2, load_model

In [2]:
val_dir = '/media/maindisk/maximos/data/hooktheory_all12_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies/'
tokenizer = CSGridMLMTokenizer(fixed_length=256)
# val_dataset = CSGridMLMDataset(val_dir, tokenizer, 512)

In [3]:
# curriculum_type = 'random'
curriculum_type = 'base2'
subfolder = 'all12'
device_name = 'cuda:2'
model = load_model(
    curriculum_type=curriculum_type,
    subfolder=subfolder,
    device_name='cuda:2',
    tokenizer=tokenizer,
    pianoroll_dim=100
)

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

26203106


In [5]:
import os
data_files = []
for dirpath, _, filenames in os.walk(val_dir):
    for file in filenames:
        if file.endswith('.xml') or file.endswith('.mxl') or file.endswith('.musicxml'):
            full_path = os.path.join(dirpath, file)
            data_files.append(full_path)
print(len(data_files))

10486


In [7]:
val_idx = 0

input_f = data_files[val_idx]

mxl_folder = 'examples_musicXML/' + subfolder + '/' + curriculum_type + '/' + '/'
midi_folder = 'examples_MIDI/' + subfolder + '/' + curriculum_type + '/' + '/'
os.makedirs(mxl_folder, exist_ok=True)
os.makedirs(midi_folder, exist_ok=True)

gen_harm, real_harm = generate_files_with_base2(
    model=model,
    tokenizer=tokenizer,
    input_f=input_f,
    mxl_folder=mxl_folder,
    midi_folder=midi_folder,
    name_suffix=0
)

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/base.py:559: UserWarning: Setting an ID that could be mistaken for a memory location is discouraged: got 128516249091680
  warnings.warn(msg)
libpng warning: iCCP: known incorrect sRGB profile
Error at line 30 col 47: no instrument found for part 'Pdc6e35c6266dd43964141c98a58ed639'
Error at line 443 col 47: no instrument found for part 'Pb992f6fdb5ab535ad8db1ad98f384705'
convert <examples_musicXML/all12/base2//gen_0.mxl>...
	to <examples_MIDI/all12/base2//gen_0.mid>
... success!
libpng warning: iCCP: known incorrect sRGB profile
Error at line 30 col 47: no instrument found for part 'P4acdb62bbf5f97d092ba9b01e7ab1e0a'
Error at line 443 col 47: no instrument found for part 'P39a991945c7a373b16bb2148145cff97'
convert <examples_musicXML/all12/base2//real_0.mxl>...
	to <examples_MIDI/all12/base2//real_0.mid>
... success!
